In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-23 13:25:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-10-23 13:25:10 (9.07 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
#reading the input file by characters
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("Number of characters in input", len(text))

Number of characters in input 1115394


In [6]:
print("first 1000 characters \n", text[:1000])

first 1000 characters 
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not 

In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("the size of vocab ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
the size of vocab  65


In [15]:
#simple encoder decoder based on char position
# also know tokenizer
encode_map = {ch:i for i, ch in enumerate(chars)} # look up table
decode_map = {i:ch for i, ch in enumerate(chars)}
encode = lambda st : [encode_map[ch] for ch in st] # lambda return everything after :. so they sould be be something returnable
decode = lambda num : ''.join([decode_map[int_num] for int_num in num])
print(encode("hi there!!"))
print(decode(encode("hi there!!")))

[46, 47, 1, 58, 46, 43, 56, 43, 2, 2]
hi there!!


In [16]:
import torch
data = torch.tensor(encode(text), dtype=torch.long) # converting 
print(data.shape, data.type)
print(data[:1000])


torch.Size([1115394]) <built-in method type of Tensor object at 0x7c8ad633f4d0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [19]:
#split train and test data
data_split_len = int(0.85*len(data))
train_data = data[:data_split_len]
val_data = data[data_split_len:]

In [20]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [26]:
x = train_data[:block_size] # input to transformer 
y = train_data[1:block_size+1] # target to transformer
for r in range(block_size):
    print(f"train data : {x[:r+1]} and next prediction {y[r]}")

train data : tensor([18]) and next prediction 47
train data : tensor([18, 47]) and next prediction 56
train data : tensor([18, 47, 56]) and next prediction 57
train data : tensor([18, 47, 56, 57]) and next prediction 58
train data : tensor([18, 47, 56, 57, 58]) and next prediction 1
train data : tensor([18, 47, 56, 57, 58,  1]) and next prediction 15
train data : tensor([18, 47, 56, 57, 58,  1, 15]) and next prediction 47
train data : tensor([18, 47, 56, 57, 58,  1, 15, 47]) and next prediction 58


In [32]:
torch.manual_seed(1337)
batch_size = 4
# get for random ints for the batch size.
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # getting a random 4 intergers
    print(ix)
    #get data for those 4 
    x = torch.stack([data[i:i+block_size] for i in ix]) # from current element to the size of block
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb) # target fot the input

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

tensor([ 21203, 291993, 823924,  59812])
inputs:
torch.Size([4, 8])
tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])
targets:
torch.Size([4, 8])
tensor([[61, 57, 10,  0, 20, 43,  1, 58],
        [41, 43, 42,  1, 58, 46, 43,  1],
        [41, 43,  8,  0,  0, 24, 17, 27],
        [33, 31, 10,  0, 25, 53, 57, 58]])
----
when input is [53] the target: 61
when input is [53, 61] the target: 57
when input is [53, 61, 57] the target: 10
when input is [53, 61, 57, 10] the target: 0
when input is [53, 61, 57, 10, 0] the target: 20
when input is [53, 61, 57, 10, 0, 20] the target: 43
when input is [53, 61, 57, 10, 0, 20, 43] the target: 1
when input is [53, 61, 57, 10, 0, 20, 43, 1] the target: 58
when input is [39] the target: 41
when input is [39, 41] the target: 43
when input is [39, 41, 43] the target: 42
when input is [39, 41, 43, 42] the target: 1
when input is [39, 41, 43

In [33]:
print(xb) # first input batch for our transformer

tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])


In [36]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embed_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):
        logits = self.token_embed_table(idx)

        loss = F.cross_entropy(logits, target)

        return logits, loss

model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

RuntimeError: Expected target size [4, 65], got [4, 8]